# Demonstration for the upstream regulation network reconstruction

This notebook demonstrates how to use BRAvo to reconstruct a regulation network with specific options and use it as a model, here to infer new knownledge from partial knowledge on nodes evolution.

In [1]:
# Imports
import time
import csv
import logging
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

## Network reconstruction

### Global options

We define below the global options regarding the execution:

- Max depth of 10 levels
- Fast reconstruction (look only at standard names)
- Decompose complexes
- Extend search by considering synonyms and removing suffixes
- Include unsinged interactions
- Use Pathway Commons' SPARQL endpoint
- Consider all data sources except MiRTarBase and MSigDB

In [2]:
# Global options

config.MAX_DEPTH = 10
config.HAS_MAX_DEPTH = True
config.FAST = False
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.UNKNOWN = True
config.SPARQL_ENDPOINT = "http://rdf.pathwaycommons.org/sparql/"
config.VERBOSE = True

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
      'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
      'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
      'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
      'intact_complex', 'msigdb', 'reach', 'wikipathways']

config.DATA_SOURCES = set(ds) - set(['mirtarbase', 'msigdb'])

pybravo_stdout = [''] * 3

### Gene names input list

Here we read the list of input gene names from a file called `input-910.csv`. This file should contain one gene name per line.

In [3]:
# Read input genes list from a file
def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Genes input list
gene_list = read_input_genes('reg-demo/data/input-910.csv')
print('{} genes in input list'.format(len(gene_list)))

910 genes in input list


### Initial reconstruction

We now call the reconstruction on the previous list of gene names.
This iterative step is performed by querying the Pathway Commons endpoint and using the results to complete the graph and prepare the next query.
It may take some time, typically between 20 and 30 minutes.

In [4]:
# Add logging handler to file for later analysis
handler = logging.FileHandler('reg-demo/std.log', mode = 'w')
handler.setFormatter(logging.Formatter('%(message)s'))
logging.getLogger().addHandler(handler)

In [5]:
%%capture --no-display pybravo_stdout_0

start_time = time.time()

# Call reconstruction
reconstructed_network = regulation.upstream_regulation(gene_list)

elapsed_time = round((time.time() - start_time), 2)

2019-12-06 13:20:19,846 - INFO: to be explored after complex decomposition ['LUM', 'CPZ', 'DCN', 'ISLR', 'FNDC1', 'THBS2', 'COL16A1', 'COL1A1', 'COL3A1', 'COL5A1', 'SSC5D', 'MMP28', 'GEM', 'DACT3', 'MRC2', 'GPR124', 'LOXL1', 'ST6GAL2', 'COL1A2', 'PRICKLE1', 'LTBP2', 'COL6A3', 'CDR2L', 'PMP22', 'EMILIN1', 'COL14A1', 'CDH11', 'C1QTNF2', 'CCDC80', 'EFEMP1', 'GLT8D2', 'GGT5', 'ANTXR1', 'SNAI1', 'TSHZ3', 'PPAPDC1A', 'ALDH1A3', 'HEPH', 'FBN1', 'ECM1', 'DPYSL3', 'SRPX', 'PODN', 'GFPT2', 'FOXL1', 'COL10A1', 'GAS1', 'ITGA11', 'GLI2', 'HAND2', 'VCAN', 'GPX8', 'POSTN', 'CRISPLD2', 'PTGIR', 'CALHM2', 'FOXF1', 'MOXD1', 'MGP', 'ASPN', 'COL12A1', 'ZNF469', 'CHST3', 'NHS', 'LAMA2', 'AEBP1', 'PCDH7', 'HTRA3', 'HDGFRP3', 'MXRA5', 'HEYL', 'SVEP1', 'MAP1A', 'SPEG', 'GLI3', 'COL6A2', 'F2R', 'FAP', 'C13orf33', 'SEMA3C', 'ALDH1L2', 'LXN', 'PHLDB1', 'PRELP', 'ADAMTS2', 'PMEPA1', 'EPHA3', 'FLNA', 'RSPO3', 'TGFB3', 'TMEM132E', 'SGCA', 'C1QTNF7', 'COL8A2', 'ADAMTS12', 'MFAP4', 'LAMA1', 'PLAT', 'CTBP2', 'COL4A2',

2019-12-06 13:20:19,848 - INFO: new synonmys to be explored:['LDC', 'SLRR2D', 'CSCD', 'DSPG2', 'PG40', 'PGII', 'PGS2', 'SLRR1B', 'HsT17563', 'Meflin', 'AGS8', 'FNDC2', 'MEL4B3', 'bA243O10.1', 'dJ322A24.1', 'TSP2', '447AA', 'FP1572', 'EDSARTH1', 'EDSC', 'OI1', 'OI2', 'OI3', 'OI4', 'EDS4A', 'EDSVASC', 'EDSC', 'EDSCL1', 'S5D-SRCRB', 'EPILYSIN', 'MM28', 'MMP-25', 'MMP-28', 'MMP25', 'KIR', 'DAPPER3', 'RRR1', 'CD280', 'CLEC13E', 'ENDO180', 'UPARAP', 'ADGRA2', 'TEM5', 'LOL', 'LOXL', 'SIAT2', 'ST6GalII', 'EDSARTH2', 'EDSCV', 'OI4', 'EPM1B', 'RILP', 'LTBP3', 'DASS', 'GPHYSD3', 'LTBP-3', 'STHAG6', 'pp6425', 'BTHLM1', 'DYT27', 'UCMD1', 'HUMPPA', 'PXMP2', 'EMI', 'EMILIN', 'gp115', 'UND', 'CAD11', 'CDHOB', 'ESWS', 'OB', 'OSF-4', 'CTRP2', 'zacrp2', 'DRO1', 'SSG1', 'URB', 'okuribin', 'DHRD', 'DRAD', 'FBLN3', 'FBNL', 'FIBL-3', 'MLVT', 'MTLV', 'S1-5', 'GGL', 'GGT 5', 'GGT-REL', 'GGTLA1', 'ATR', 'GAPO', 'TEM8', 'SLUGH2', 'SNA', 'SNAH', 'SNAIL', 'SNAIL1', 'dJ710H13.1', 'TSH3', 'ZNF537', 'PLPP4', 'DPPL2',

2019-12-06 13:21:22,476 - INFO: 		removing suffix  complex for NFX1/SIN3/HDAC complex
2019-12-06 13:21:22,478 - INFO: 				 --> NFX1/SIN3/HDAC
2019-12-06 13:23:51,875 - INFO: 		removing suffix  complex for tungsten carbide/Cobalt complex
2019-12-06 13:23:51,876 - INFO: 				 --> tungsten carbide/Cobalt
2019-12-06 13:23:51,877 - INFO: 		removing suffix  protein for DCN protein
2019-12-06 13:23:51,878 - INFO: 				 --> DCN
2019-12-06 13:23:51,879 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 13:23:51,880 - INFO: 				 --> NOG
2019-12-06 13:23:51,881 - INFO: 		removing suffix  protein for DCN protein
2019-12-06 13:23:51,883 - INFO: 				 --> DCN
2019-12-06 13:23:51,884 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 13:23:51,885 - INFO: 				 --> NOG
2019-12-06 13:23:51,885 - INFO: 		removing suffix  mRNA for LUM mRNA
2019-12-06 13:23:51,886 - INFO: 				 --> LUM
2019-12-06 13:23:51,887 - INFO: 		removing suffix  protein for INS protein
2019-12-06 13:23:51,887 - INF

2019-12-06 13:23:57,421 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 13:23:57,421 - INFO: 				 --> COL1A1
2019-12-06 13:23:57,422 - INFO: 		removing suffix  protein for IL6 protein
2019-12-06 13:23:57,423 - INFO: 				 --> IL6
2019-12-06 13:23:57,423 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 13:23:57,423 - INFO: 				 --> COL1A1
2019-12-06 13:23:57,424 - INFO: 		removing suffix  protein for DEFA1 protein
2019-12-06 13:23:57,424 - INFO: 				 --> DEFA1
2019-12-06 13:23:57,425 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 13:23:57,425 - INFO: 				 --> COL1A1
2019-12-06 13:23:57,426 - INFO: 		removing suffix  protein for LEP protein
2019-12-06 13:23:57,426 - INFO: 				 --> LEP
2019-12-06 13:23:57,427 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2019-12-06 13:23:57,428 - INFO: 				 --> COL1A1
2019-12-06 13:23:57,428 - INFO: 		removing suffix  protein for ABL1 protein
2019-12-06 13:23:57,429 - INFO: 				 --> ABL1
2019-12-06 13:23:57,429 - INFO: 		

2019-12-06 13:24:05,936 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2019-12-06 13:24:05,937 - INFO: 				 --> PMP22
2019-12-06 13:24:05,939 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 13:24:05,940 - INFO: 				 --> NOG
2019-12-06 13:24:05,942 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2019-12-06 13:24:05,946 - INFO: 				 --> COL6A3
2019-12-06 13:24:05,947 - INFO: 		removing suffix  protein for INS protein
2019-12-06 13:24:05,949 - INFO: 				 --> INS
2019-12-06 13:24:05,950 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2019-12-06 13:24:05,951 - INFO: 				 --> PMP22
2019-12-06 13:24:05,952 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 13:24:05,954 - INFO: 				 --> TGFB1
2019-12-06 13:24:05,956 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2019-12-06 13:24:05,957 - INFO: 				 --> COL6A3
2019-12-06 13:24:05,958 - INFO: 		removing suffix  protein for pirinixic acid/PPARA protein complex
2019-12-06 13:24:05,959 - INFO: 				 --> pirinixic acid/PPAR

...

2019-12-06 14:33:55,560 - INFO: 				 --> TP53 gene
2019-12-06 14:33:55,561 - INFO: 		removing suffix  gene for TP53 gene
2019-12-06 14:33:55,561 - INFO: 				 --> TP53
2019-12-06 14:33:55,562 - INFO: 		removing suffix  protein for TP53 protein
2019-12-06 14:33:55,565 - INFO: 				 --> TP53
2019-12-06 14:33:55,568 - INFO: 		removing suffix  protein for PRKN protein
2019-12-06 14:33:55,569 - INFO: 				 --> PRKN
2019-12-06 14:33:55,570 - INFO: 		removing suffix  protein for TP53 protein
2019-12-06 14:33:55,572 - INFO: 				 --> TP53
2019-12-06 14:33:55,575 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 14:33:55,576 - INFO: 				 --> NOG
2019-12-06 14:33:55,577 - INFO: 		removing suffix  mRNA for EGR1 mRNA
2019-12-06 14:33:55,578 - INFO: 				 --> EGR1
2019-12-06 14:33:55,580 - INFO: 		removing suffix  protein for ESR2 protein
2019-12-06 14:33:55,581 - INFO: 				 --> ESR2
2019-12-06 14:33:55,582 - INFO: 		removing suffix  mRNA for EGR1 mRNA
2019-12-06 14:33:55,583 - INFO: 				 --

2019-12-06 14:34:01,365 - INFO: 				 --> NOG
2019-12-06 14:34:01,366 - INFO: 		removing suffix  mRNA for KLF4 mRNA
2019-12-06 14:34:01,368 - INFO: 				 --> KLF4
2019-12-06 14:34:01,370 - INFO: 		removing suffix  protein for NOG protein
2019-12-06 14:34:01,372 - INFO: 				 --> NOG
2019-12-06 14:34:01,373 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:34:01,375 - INFO: 				 --> PPARG
2019-12-06 14:34:01,379 - INFO: 		removing suffix  protein for INS protein
2019-12-06 14:34:01,381 - INFO: 				 --> INS
2019-12-06 14:34:01,384 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:34:01,386 - INFO: 				 --> PPARG
2019-12-06 14:34:01,390 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 14:34:01,392 - INFO: 				 --> TGFB1
2019-12-06 14:34:01,395 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:34:01,397 - INFO: 				 --> PPARG
2019-12-06 14:34:01,400 - INFO: 		removing suffix  protein for IFNG protein
2019-12-06 14:34:01,403 - I

2019-12-06 14:36:42,931 - INFO: 				 --> NOG
2019-12-06 14:36:42,934 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 14:36:42,935 - INFO: 				 --> MLXIPL
2019-12-06 14:36:42,937 - INFO: 		removing suffix  protein for INS protein
2019-12-06 14:36:42,938 - INFO: 				 --> INS
2019-12-06 14:36:42,939 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 14:36:42,940 - INFO: 				 --> MLXIPL
2019-12-06 14:36:42,942 - INFO: 		removing suffix  protein for INS protein
2019-12-06 14:36:42,943 - INFO: 				 --> INS
2019-12-06 14:36:42,944 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 14:36:42,945 - INFO: 				 --> MLXIPL
2019-12-06 14:36:42,947 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 14:36:42,948 - INFO: 				 --> TGFB1
2019-12-06 14:36:42,949 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2019-12-06 14:36:42,950 - INFO: 				 --> MLXIPL
2019-12-06 14:37:07,709 - INFO: 		removing suffix  protein for TNF protein
2019-12-06 14:37:07,712 - INFO: 				

2019-12-06 14:39:03,809 - INFO: 				 --> Estradiol/ESR2 complex
2019-12-06 14:39:03,811 - INFO: 		removing suffix  complex for Estradiol/ESR2 complex
2019-12-06 14:39:03,812 - INFO: 				 --> Estradiol/ESR2
2019-12-06 14:39:03,814 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 14:39:03,816 - INFO: 				 --> MYC
2019-12-06 14:39:03,820 - INFO: 		removing suffix  mutant form for STK11 mutant form
2019-12-06 14:39:03,822 - INFO: 				 --> STK11
2019-12-06 14:39:03,825 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 14:39:03,827 - INFO: 				 --> MYC
2019-12-06 14:39:03,833 - INFO: 		removing suffix  protein for ESR1 protein
2019-12-06 14:39:03,837 - INFO: 				 --> ESR1
2019-12-06 14:39:03,840 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-06 14:39:03,842 - INFO: 				 --> MYC
2019-12-06 14:39:03,848 - INFO: 		removing suffix  protein for TNF protein
2019-12-06 14:39:03,851 - INFO: 				 --> TNF
2019-12-06 14:39:03,854 - INFO: 		removing suffix  mRNA for MYC mRNA
2019-12-0

2019-12-06 14:39:09,518 - INFO: 				 --> PPARG
2019-12-06 14:39:09,520 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 14:39:09,521 - INFO: 				 --> TGFB1
2019-12-06 14:39:09,522 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:39:09,523 - INFO: 				 --> PPARG
2019-12-06 14:39:09,524 - INFO: 		removing suffix  protein for TGFB1 protein
2019-12-06 14:39:09,525 - INFO: 				 --> TGFB1
2019-12-06 14:39:09,525 - INFO: 		removing suffix  mRNA for AHR mRNA
2019-12-06 14:39:09,526 - INFO: 				 --> AHR
2019-12-06 14:39:09,527 - INFO: 		removing suffix  protein for IFNG protein
2019-12-06 14:39:09,527 - INFO: 				 --> IFNG
2019-12-06 14:39:09,528 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:39:09,529 - INFO: 				 --> PPARG
2019-12-06 14:39:09,531 - INFO: 		removing suffix  protein for IFNG protein
2019-12-06 14:39:09,532 - INFO: 				 --> IFNG
2019-12-06 14:39:09,533 - INFO: 		removing suffix  protein for PPARG protein
2019-12-06 14:39:09,53

In [6]:
print("--- Upstream regulation network reconstructed in {} seconds ---".format(elapsed_time))

--- Upstream regulation network reconstructed in 4772.73 seconds ---


In [7]:
%%capture --no-display pybravo_stdout_1

# Build digraph representation in memory
G = bravo_main.build_nx_digraph(reconstructed_network)

bravo_main.write_to_SIF(G, 'reg-demo/out.sif')
bravo_main.write_provenance(G, 'reg-demo/out-provenance.csv')

### Synonym-based unification

We now perform a synonym-unification, which consists in fusing nodes together if they are synonyms.
This process is useful because of the high heterogeneity of the data sources that compose Pathway Commons.
It typically takes a few seconds.

In [8]:
%%capture --no-display pybravo_stdout_2

start_time = time.time()
# Perform unification
G_prime = util.fast_reg_network_unification(G, util.index_syn)
elapsed_time = round((time.time() - start_time), 2)
print("--- Network simplification in %s seconds ---" % elapsed_time)

# Write to files
bravo_main.write_to_SIF(G_prime, 'reg-demo/out-unified.sif')
bravo_main.write_provenance(G_prime, 'reg-demo/out-unified-provenance.csv')

# Count nodes/edges
print('Nodes after simplification = ' + str(len(G_prime.nodes())))
print('Edges after simplification = ' + str(len(G_prime.edges())))

# Compute centrality
print(bravo_main.get_centrality_as_md(G_prime))

2019-12-06 14:39:52,846 - INFO: 		merging node TIF2 into node NCOA2
2019-12-06 14:39:52,880 - INFO: 		merging node BRM into node SMARCA2
2019-12-06 14:39:52,911 - INFO: 		merging node PCAF into node KAT2B
2019-12-06 14:39:52,955 - INFO: 		merging node BK2 into node BDKRB2
2019-12-06 14:39:52,980 - INFO: 		merging node COX-2 into node PTGS2
2019-12-06 14:39:53,008 - INFO: 		merging node TCF1 into node HNF1A
2019-12-06 14:39:53,032 - INFO: 		merging node HNF4A into node HNF1A
2019-12-06 14:39:53,057 - INFO: 		merging node TCF1 into node HNF1A
2019-12-06 14:39:53,086 - INFO: 		merging node IL8 into node CXCL8
2019-12-06 14:39:53,109 - INFO: 		merging node ATF1 into node GDNF
2019-12-06 14:39:53,205 - INFO: 		merging node ATF2 into node GDNF
2019-12-06 14:39:53,236 - INFO: 		merging node AGT into node AGXT
2019-12-06 14:39:53,271 - INFO: 		merging node DAT into node SLC6A3
2019-12-06 14:39:53,295 - INFO: 		merging node maspin into node SERPINB5
2019-12-06 14:39:53,331 - INFO: 		merging nod

2019-12-06 14:39:57,534 - INFO: 		merging node SELP into node SELENOP
2019-12-06 14:39:57,571 - INFO: 		merging node MIZ-1 into node ZBTB17
2019-12-06 14:39:57,595 - INFO: 		merging node CBF into node CEBPZ
2019-12-06 14:39:57,705 - INFO: 		merging node PC into node PROC
2019-12-06 14:39:57,738 - INFO: 		merging node CaM into node CALM3
2019-12-06 14:39:57,781 - INFO: 		merging node p107 into node RBL1
2019-12-06 14:39:57,807 - INFO: 		merging node ATF2 into node GDNF
2019-12-06 14:39:57,849 - INFO: 		merging node CRIP1 into node CNRIP1
2019-12-06 14:39:57,872 - INFO: 		merging node TIP60 into node KAT5
2019-12-06 14:39:57,893 - INFO: 		merging node NFATc into node NFATC1
2019-12-06 14:39:57,920 - INFO: 		merging node FGF2 into node FGF13
2019-12-06 14:39:57,950 - INFO: 		merging node NIR into node NOC2L
2019-12-06 14:39:57,970 - INFO: 		merging node TIP49B into node RUVBL2
2019-12-06 14:39:58,066 - INFO: 		merging node p300 into node EP300
2019-12-06 14:39:58,094 - INFO: 		merging nod

### Saving output

Here we simply save all output provided by pyBRAvo during the reconstruction into a texte file.

In [9]:
all_stdout = '\n'.join([s.stdout for s in [pybravo_stdout_0, pybravo_stdout_1, pybravo_stdout_2]])
print(all_stdout)

Explored 16 regulators
Explored 16 regulators
Explored 26 regulators
Explored 26 regulators
Explored 29 regulators
Explored 33 regulators
Explored 37 regulators
Explored 41 regulators
Explored 43 regulators
Explored 44 regulators
Explored 44 regulators
Explored 51 regulators
Explored 57 regulators
Explored 57 regulators
Explored 57 regulators
Explored 57 regulators
Explored 58 regulators
Explored 59 regulators
Explored 78 regulators
Explored 78 regulators
Explored 80 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 82 regulators
Explored 82 regulators
Explored 82 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 91 regulators
Explored 91 regulators
Explored 91

In [10]:
with open('reg-demo/std.out', 'w') as outfile:
    print(all_stdout, file = outfile)

### Recovering logs

Here we read all logs saved to a file during the reconstruction for a later analysis.

In [11]:
with open('reg-demo/std.log', 'r') as logfile:
    pybravo_log_0 = logfile.read()

### Statistics extraction

Here we call several Bash and Python scripts on the results to extracts statistics on the provenance, coverage and content of the network. Some data are summed up in file `cytoscape/edges-provenance.csv` that can be loaded in Cytoscape and applied to the edges of the network.

In [17]:
%%bash
cd reg-demo

# Extract genes of the unified and non-unified results
sh scripts/get-genes-from-sif.sh out-unified.sif > out-unified.sif.genes
# Provenance of all edges in unified result
bash scripts/pybravo-provenances.sh out-unified-provenance.csv > provenance-unified.txt
# Provenance of signed edges only in unified result
bash scripts/pybravo-provenances.sh <(grep -v 'UNKNOWN' out-unified-provenance.csv) > provenance-unified-signed.txt

# Coverage
python3 scripts/compute_coverage.py out-unified.sif data/input-910.csv ../Homo_sapiens.gene_info > coverage.txt

# Provenance CSV for Cytoscape
mkdir cytoscape
echo "edge name,provenance" > cytoscape/edges-provenance.csv
sed 's/\t\(.*\)\t/ (\1) /;' out-unified-provenance.csv >> cytoscape/edges-provenance.csv

In [18]:
%%bash --no-raise-error
cd reg-demo
bash scripts/stats-2.sh "out-unified.sif" "std.log"

Nodes: 1630 out-unified.sif.genes
Edges: 4283 out-unified.sif
 - signed: 3715
 - unsigned: 0
 - PART_OF: 568

Coverage: 704 genes are present over 910

Complexes: 299
Complexes involving small molecules: 111


## Using Iggy to make predictions

In this part we use the previously extracted network as a model in order to make predictions on it, using the tool [Iggy](https://github.com/bioasp/iggy).

Using some preliminary observations on some nodes, consisting in signs “$+$” (observed up-regulation) and “$-$” (observed down-regulation), Iggy tries to infer new information on the other nodes which can thus also be predicted as “$+$” or “$-$”. This call relies on Bash and Ptyhon scripts.

For this step, you need Iggy 1.4.1 installed, for instance using pip:

> ```pip install iggy==1.4.1```

Once Iggy is installed, please provide the path to it below.
Please note that this can change according to the installed sofware in your machine.
For instance, the default when only pip installed is `~/.local/bin/iggy.py`.
However, if Conda or Miniconda is installed, the default becomes `~/(path-to-conda-environment)/bin/iggy.py`.

In [19]:
#iggy_command = "${HOME}/.local/bin/iggy.py"
iggy_command = "${CONDA_PREFIX}/bin/iggy.py"

In [20]:
# Interpret Iggy command line
iggy_command = !echo "$iggy_command"
iggy_command = iggy_command[0]
print(iggy_command)

/home/max/miniconda3/envs/pybravo/bin/iggy.py


The scripts below call Iggy, with some data preparation beforehand and a comparison of the result with the original ICGC experimental data afterwards.

The printed output sums up the results of the prediction, and the consistency of these results with the initial ICGC expression data (matching or not matching predictions).

The full result can be found in file `reg-demo/cytoscape/nodes-iggy.csv` in a format that can be fed to Cytoscape in order to label nodes with their observation/prediction sign.

In [21]:
%%bash -s "$iggy_command"
export LC_ALL=C
# Make folders for execution results
cd reg-demo
mkdir iggy
cd iggy
mkdir result

# Create graph (SIF) file (removes unsigned edges)
bash ../scripts/format-iggy.sh ../out-unified.sif | grep -v 'UNKNOWN' > out-unified-iggy.sif

# Create observations (OBS) file
bash ../scripts/construct-inputs.sh out-unified-iggy.sif ../data/updownreg-v4.3-noinputs.obs > updownreg.obs

# Extract true observations (actually in network)
bash ../scripts/get-genes-from-sif.sh out-unified-iggy.sif > out-unified-iggy.sif.genes 
grep -w -f out-unified-iggy.sif.genes updownreg.obs > updownreg_true.obs

# Run Iggy workflow
bash ../scripts/workflow-iggy.sh --iggy-command $1 updownreg_true.obs out-unified-iggy.sif ../data/icgc.syn.csv 0 0 result/iggy.out > result/out-unified.tsv
cp result/out-unified.tsv ../cytoscape/nodes-iggy.csv

# Output stats
bash ../scripts/stats-result.sh --en --skip0 result/out-unified.tsv out-unified-iggy.sif > result/stats.txt
cat result/stats.txt

Total number of nodes: 1630
Total number of edges: 4283
Number of observations: 654
Number of predictions: 73
  positive (+): 37
  negative (-): 31
  no-change (0): 0
  may-up (NOT-): 0
  may-down (NOT+): 0
  change (CHANGE/NOT0): 5

Predictions found in ICGC data: 55
    +: 26
    -: 27
    0: 0
  coherent: 22
    +: 15
    -: 7
  weakly coherent: 2
    NOT-: 0
    NOT+: 0
    CHANGE: 2
  not coherent: 31
    predicted +: 11
      (should be -)
     predicted -: 20
      (should be +)
     predicted NOT-/NOT+/CHANGE: 0
